# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
import os
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con poda por factibilidad.
- BT-O: Backtracking con poda por factibilidad y optimalidad.
- DP: Programación dinámica.

In [3]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./negociospormedio", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

In [4]:
def agregar_experimentos(metodo, dataframe):
    counter_n = 1
    n_a_agregar = 20
    while counter_n < 50:
        for n in range(0, 999):
            fila_n = dataframe.iloc[n];
            if fila_n["n"] == n_a_agregar and counter_n < 50:
                experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["M"], metodo, fila_n["archivo"]])
                if counter_n % 5 == 0:
                    n_a_agregar = n_a_agregar + 1
                counter_n = counter_n + 1
                    
                    
# Se toman varias instancias de programación dinamica para armar los graficos de HEATMAP con mayor visualizacion.
# Al ejecutarse en menor tiempo. Podemos permitirnos realizar mayor ejecución del programa con mayor cantidad de datos
def agregar_experimentos_heatmap(metodo, dataframe):
    counter_n = 1
    n_a_agregar = 20
    while counter_n < 300:
        for n in range(0, 999):
            fila_n = dataframe.iloc[n];
            if fila_n["n"] == n_a_agregar and counter_n < 300:
                experimentos_heatmap.append([fila_n["dataset"], fila_n["n"], fila_n["M"], metodo, fila_n["archivo"]])
                if counter_n % 30 == 0:
                    n_a_agregar = n_a_agregar + 1
                counter_n = counter_n + 1

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [5]:
dfs={}
dfs_heatmap={}
casos=["mrandommillon", "random","MSumaCSobre2","MSumaCSobre4","MNoRestringe","MSumaC","beneficioCreciente","mrandom"]
experimentos = []
experimentos_heatmap = []
metodos=["FB","BT","BT-O","DP"]


for caso in casos:
    dfs[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")
    

for metodo in metodos:
    for caso in casos:
        agregar_experimentos(metodo, dfs[caso])

        
for caso in casos:
    dfs_heatmap[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")
    agregar_experimentos_heatmap("DP", dfs_heatmap[caso])
    
    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

De todas las instancias, tomamos las que tienen 20 o mas negocios para poder efectuar un analisis entre tiempos comparables. En caso contrario, se colapsan y no se obtiene mucha información.

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [9]:
columnas = ["dataset", "n", "M", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    display('Method: ' + str(experimento[3]))
    display('Path: ' + str(experimento[4]))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 1568/1568'

'Method: DP'

'Path: instancias/mrandom/NPM-mrandom-266.txt'

In [8]:
columnas = ["dataset", "n", "M", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_heatmap:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_heatmap)))
    display('Method: ' + str(experimento[3]))
    display('Path: ' + str(experimento[4]))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    for i in range(0, T):
        tiempos.append(correr_experimento(experimento[3], experimento[4]));
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_heatmap.csv", index=False, header=True);

'Experimento: 2392/2392'

'Method: DP'

'Path: instancias/mrandom/NPM-mrandom-797.txt'